In [0]:
from __future__ import print_function

In [0]:
max_features = 12500

# cut texts after this number of words (among top max_features most common words)
maxlen = 128
batch_size = 32

In [3]:
from keras.datasets import imdb

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Using TensorFlow backend.


Loading data...
25000 train sequences
25000 test sequences


In [4]:
from keras.preprocessing import sequence

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen,padding='post')
x_test = sequence.pad_sequences(x_test, maxlen=maxlen,padding='post')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 128)
x_test shape: (25000, 128)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.regularizers import l2,l1_l2


In [6]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5,
    kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
    recurrent_regularizer=l2(1e-4),
    bias_regularizer=l2(1e-4),
    activity_regularizer=l2(1e-5),
    return_sequences=True))
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.3,
    kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4),
    recurrent_regularizer=l2(1e-4),
    bias_regularizer=l2(1e-4),
    activity_regularizer=l2(1e-5)))
model.add(Dense(1, activation='sigmoid'))



Build model...


In [0]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1600000   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,752,225
Trainable params: 1,752,225
Non-trainable params: 0
_________________________________________________________________


In [9]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))


Train...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 253s 10ms/step - loss: 0.6114 - accuracy: 0.7148 - val_loss: 0.4969 - val_accuracy: 0.8114
Epoch 2/15
25000/25000 [==============================] - 265s 11ms/step - loss: 0.5115 - accuracy: 0.8024 - val_loss: 0.4802 - val_accuracy: 0.8230
Epoch 3/15
25000/25000 [==============================] - 264s 11ms/step - loss: 0.4890 - accuracy: 0.8208 - val_loss: 0.4892 - val_accuracy: 0.8166
Epoch 4/15
25000/25000 [==============================] - 265s 11ms/step - loss: 0.4340 - accuracy: 0.8535 - val_loss: 0.4895 - val_accuracy: 0.8308
Epoch 5/15
25000/25000 [==============================] - 259s 10ms/step - loss: 0.3777 - accuracy: 0.8809 - val_loss: 0.4303 - val_accuracy: 0.8554
Epoch 6/15
25000/25000 [==============================] - 265s 11ms/step - loss: 0.2936 - accuracy: 0.9177 - val_loss: 0.4038 - val_accuracy: 0.8593
Epoch 7/15
25000/25000 [==============================] 

In [10]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 43s 2ms/step
Test score: 0.6235083810043335
Test accuracy: 0.842519998550415


In [0]:
model.save('20-05-17-2100.h5')